## _Computer Vision_
### Lab 5, _Image Interpolation_.
#### >> Topics  :
* ##### Interpolation Methods
* ##### Inpainting
* ##### Image Pyramids
* ##### Edges vs Contours
* ##### Hough Space
* ##### Hough Circles Detector

##### >> Note  :
> ##### to close image windows smoothly please **press Esc** on your keyboard, **don't close** it directly by clicking on 'X' to avoid kernel interruption.

In [8]:
import cv2 as cv
import numpy as np

### Section 0, _Helper Functions_:
> ##### This cell contains some helper function such as error_handler, please run it without any modification.

In [2]:
# please don't modify this code.
def show_text_window(titles):
    black = np.zeros((len(titles) * 150, 400))
    for idx, t in enumerate(titles):
        place = idx + 1
        cv.putText(black, t, (10, place * 100),
                   cv.FONT_HERSHEY_SIMPLEX, 0.75, (200, 0, 200), 1, 2)
    cv.imshow("Values", black)


def get_updated_value(key, value, **kwargs):
    if key == ord('+'):
        return value, 0
    elif key == ord('-'):
        return -value, 0
    elif key == ord('*'):
        return 0, kwargs.get('value2', value)
    elif key == ord('/'):
        return 0, -kwargs.get('value2', value)
    elif key == 27:
        raise Exception('Terminated by user!')
    else:
        return 0, 0


def error_handler(func):
    def wrapper(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except Exception as ex:
            cv.destroyAllWindows()
            print(f'{ex}')

    return wrapper

### Section 1, _Interpolation Methods_:
* ##### Interpolation methods are used to estimate pixel values at non-integer coordinates when resizing images.
* ##### The choice of interpolation method affects the quality of the resized image.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html)

In [6]:
@error_handler
def apply_resize_with_interpolation(path, type='linear'):
    img = cv.imread(path)
    cv.imshow('Image', img)

    scale_ratio = 1
    accum = 0
    while True:
        dimentions = img.shape
        if type == 'nearest':
            interpolation_tp = cv.INTER_NEAREST
        elif type == 'cubic':
            interpolation_tp = cv.INTER_CUBIC
        else:
            interpolation_tp = cv.INTER_LINEAR

        # resize with interpolation
        interpolated = cv.resize(img, (int(dimentions[1] * scale_ratio), int(dimentions[0] * scale_ratio)),
                                 interpolation=interpolation_tp)
        cv.imshow(f'{type} Interpolation', interpolated)

        # show text
        show_text_window([f'Scale: {scale_ratio}'])

        # process input.
        k = cv.waitKey(0)
        uv, _ = get_updated_value(k, 2)
        accum += uv
        if accum < 0:
            scale_ratio = 1 / abs(accum)
        elif accum == 0:
            continue
        else:
            scale_ratio = accum


path = '../Images/Lenna.jpg'
apply_resize_with_interpolation(path)

Terminated by user!


### Section 2, _Inpainting_:
* ##### It's Used in image processing and computer vision to fill in-missing or damaged parts of an image with plausible content.
* ##### The goal of inpainting is to restore or complete an image by estimating the missing information based on the surrounding image context
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/df/d3d/tutorial_py_inpainting.html)

In [21]:
@error_handler
def inpainting(img_path, mask_path):
    image = cv.imread(img_path)
    mask = cv.imread(mask_path, 0)

    # make sure that the dimensions are equal.
    assert (image.shape[0], image.shape[1]) == mask.shape

    # Radius of a circular neighborhood of each point inpainted that is considered by the algorithm.
    inpaint_radius = 1
    while True:
        inpainted_img = cv.inpaint(image, mask, inpaint_radius, cv.INPAINT_TELEA)

        cv.imshow("OriginalImage", image)
        cv.imshow('mask', mask)
        cv.imshow('Result', inpainted_img)

        # show text
        show_text_window([f'Radius: {inpaint_radius}'])

        # process input.
        k = cv.waitKey(0)
        uv, _ = get_updated_value(k, 1)

        # increase / decrease
        inpaint_radius += uv


img_path = "../Images/messi.jpg"
mask_path = "../Images/mask.png"

inpainting(img_path, mask_path)

Terminated by user!


### Section 3, _Image Pyramids_:
* ##### They are a multi-scale representation of an image at different resolutions.
* ##### They are constructed by repeatedly applying a smoothing and down-sampling operation to the original image to generate a series of images at different scales.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d4/d1f/tutorial_pyramids.htmll)

In [8]:
@error_handler
def pyramid(path):
    img = cv.imread(path)
    lower_reso = cv.pyrDown(img)
    upper_reso = cv.pyrUp(img)
    show_text_window([f'Image Dimensions: {img.shape}', f'Lower Dimensions: {lower_reso.shape}',
                      f'Upper Dimensions: {upper_reso.shape}'])

    cv.imshow('Original Image', img)
    cv.imshow('Lower', lower_reso)
    cv.imshow('Upper', upper_reso)

    cv.waitKey(0)
    cv.destroyAllWindows()


path = "../Images/home.jpg"
pyramid(path)

### Section 4, _Edges vs Contours_:
* ##### Contours are used to identify and represent the boundaries or outlines of objects in an image.
* ##### They provide information about the shapes and spatial relationships of objects within the image.
* ##### The output of contour detection is a set of curves that represent the boundaries of objects or regions in the image.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html)

In [5]:
@error_handler
def contours(path):
    image = cv.imread(path)
    image_copy = image
    thresh = 100
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    while True:
        _, binary = cv.threshold(gray, thresh, 255, cv.THRESH_BINARY)
        cv.imshow('Gray Image', gray)
        cv.imshow('Binary Image', binary)

        contours, _ = cv.findContours(binary, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        image_ = cv.drawContours(image, contours, -1, (0, 255, 0), 2)

        cv.imshow("Input Image", image_copy)
        cv.imshow("Contour Detected Image", image_)

        show_text_window([f'Threshold: {thresh}'])

        k = cv.waitKey(0)
        uv, _ = get_updated_value(k, 25)
        thresh += uv


path = "../Images/sudoku.jpg"
contours(path)

Terminated by user!


### Section 5, _Hough Space_:
* ##### It's a mathematical concept to detect and represent geometric shapes, patterns, and features in an image.
* ##### Hough Transform, a technique originally developed for line detection but later extended to detect other shapes such as circles and ellipses.
* ##### Hough space plays a crucial role in finding and representing these shapes in a parametric form.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d9/db0/tutorial_hough_lines.html)

In [20]:
def circles(path):
    image = cv.imread(path)
    image = cv.medianBlur(image, 5)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, 20, param1=100, param2=25, minRadius=0, maxRadius=0)
    circles = np.uint16(np.around(circles))

    for circle in circles[0]:
        *center, radius = circle
        cv.circle(image, center, radius, (0, 255, 0), 2)
        cv.circle(image, center, 2, (0, 0, 255), 3)
        cv.imshow('Detected Circles', image)
        cv.waitKey(0)

    cv.waitKey(0)
    cv.destroyAllWindows()


path = "../Images/coins.jpg"
circles(path)

## _The End_.